# Ollama PDF RAG Notebook

## Import Libraries


In [18]:
%pip install "unstructured[pdf]"

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\skp68\\PycharmProjects\\RAGOLLAMA\\.venv\\Lib\\site-packages\\torch\\contrib\\_tensorboard_vis.py'
Check the permissions.


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pikepdf-9.10.2-cp312-cp312-win_amd64.whl.metadata (8.2 kB)
  Using cached google_cloud_vision-3.10.2-py3-none-any.whl.metadata (9.6 kB)
  Using cached effdet-0.4.1-py3-none-any.whl.metadata (33 kB)
  Using cached unstructured_inference-1.0.5-py3-none-any.whl.metadata (5.3 kB)
  Using cached matplotlib-3.10.5-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached timm-1.0.19-py3-none-any.whl.metadata (60 kB)
  Using cached torchvision-0.22.1-cp312-cp312-win_amd64.whl.metadata (6.1 kB)
  Using cached google_api_core-2.25.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached grpcio_status-1.74.0-py3-none-any.whl.metadata (1.1 kB)
Using cached unstructured_inference-1.0.5-py3-none-any.whl (48 kB)
Using cached torch-2.7.1-cp312-cp312-win_amd64.whl (216.1 MB)
Using cached effdet-0.4.1-py3-none-any.whl (112 kB)
Using cached timm-1.0.19-py3-none-any.whl (2.5 MB)
Using cached google_cloud_vision-3.10.2-py3-non

In [ ]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF

In [27]:
from PyPDF2 import PdfReader

def load_pdf(file_path):
    with open(file_path, "rb") as f:
        reader = PdfReader(f)
        return " ".join([page.extract_text() for page in reader.pages if page.extract_text()])

local_path = r"C:\Users\skp68\PycharmProjects\RAGOLLAMA\data\AI Training Document.pdf"
data = load_pdf(local_path)
print("Loaded:", len(text), "chars")

Loaded: 68688 chars


## Split text into chunks

In [29]:
# Split text into chunks
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Assume 'data' is a string containing the full PDF text
if isinstance(data, str):
    # Wrap the text in a Document object
    data = [Document(page_content=data)]

# Now split safely
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)

print(f"Text split into {len(chunks)} chunks")

Text split into 87 chunks


## Create vector database

In [30]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Vector database created successfully


## Set up LLM and Retrieval

In [ ]:
# Set up LLM and retrieval
local_model = "llama3.2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [32]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [ ]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [34]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [35]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [42]:
# Example 1
chat_with_pdf("What is the summary of this document?")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


The main idea of this document appears to be a User Agreement for eBay's marketplace, outlining the terms and conditions that users must agree to in order to use the platform. It covers various aspects such as listing policies, dispute resolution, intellectual property, and user responsibilities. The document is long and detailed, indicating that it serves as a comprehensive agreement between eBay and its users.

The document appears to be a User Agreement between eBay and its users. The summary can be condensed as follows:

* eBay's role is limited in transactions between buyers and sellers.
* eBay provides guidance, but users are responsible for following it.
* Listings may not appear in search results due to various factors (e.g., buyer location, item format, price).
* Users release any claims against eBay from disputes or claims.
* Sellers are responsible for their listings, including compliance with public or private recalls.
* Buyers and sellers agree to abide by the Rules and policies for buyers/sellers.

Note that this summary is based only on the provided context (the "section above") and may not be a comprehensive overview of the entire document.

In [43]:
# Example 2
chat_with_pdf("tell me about the returning policies?")

According to the provided context, here are some key points about eBay's returning policies:

1. Sellers can create rules to automate replacements, returns, and refunds under certain circumstances.
2. For new sellers, listings where returns are accepted will have a default rule that automates the return process.
3. If a seller has set up this automation, an eBay-generated return shipping label will be provided to the buyer.
4. Sellers must comply with eBay's returns policy.
5. Buyers generally do not have the right to cancel an order, but they can request cancellation within a specified time period and according to eBay's Order Cancellation Policy.
6. If a seller accepts the request, they have 3 days to respond.
7. In certain cases, if a buyer is responsible for return shipping costs, they may elect to use an eBay-generated shipping label or purchase one directly from a carrier.
8. Sellers are responsible for paying the cost of an eBay-generated return shipping label.

Additionally, the following points relate to returns and refunds:

* If a transaction is canceled after payment has been completed, eBay may issue a refund to the buyer on the seller's behalf and charge the seller for the amount of the refund.
* Sellers can also be charged for the cost of return shipping labels and/or other reasonable fees from sellers when certain conditions are met (e.g., using an eBay-generated label, returns being automated, etc.).
* The eBay Payment Entities may invoice sellers for these charges and collect such charges as described in the Payments Terms of Use.
* Refund timing may vary in accordance with the rules of third parties, such as credit and debit card networks.

It's worth noting that there are also certain restrictions on returns and refunds, including:

* Sellers must comply with eBay's Money Back Guarantee or other applicable terms.
* The buyer is responsible for paying any fees associated with a refund (e.g., PayPal fees).
* Refunds may not be issued in all cases, depending on the specific circumstances of the return.

## Clean up (optional)

In [ ]:
# Optional: Clean up when done 
vector_db.delete_collection()
print("Vector database deleted successfully")